In [1]:
import nest_asyncio
nest_asyncio.apply()

In [22]:
%set_env OPENAI_API_KEY=

env: OPENAI_API_KEY=


In [3]:
# TODO: abstract all of this into a function that takes in a PDF file name 

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, SummaryIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.vector_stores import MetadataFilters, FilterCondition
from typing import List, Optional

def get_doc_tools(
    file_path: str,
    name: str,
) -> str:
    """Get vector query and summary query tools from a document."""

    # load documents
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(documents)
    vector_index = VectorStoreIndex(nodes)
    
    def vector_query(
        query: str, 
        page_numbers: Optional[List[str]] = None
    ) -> str:
        """Use to answer questions over the MetaGPT paper.
    
        Useful if you have specific questions over the MetaGPT paper.
        Always leave page_numbers as None UNLESS there is a specific page you want to search for.
    
        Args:
            query (str): the string query to be embedded.
            page_numbers (Optional[List[str]]): Filter by set of pages. Leave as NONE 
                if we want to perform a vector search
                over all pages. Otherwise, filter by the set of specified pages.
        
        """
    
        page_numbers = page_numbers or []
        metadata_dicts = [
            {"key": "page_label", "value": p} for p in page_numbers
        ]
        
        query_engine = vector_index.as_query_engine(
            similarity_top_k=2,
            filters=MetadataFilters.from_dicts(
                metadata_dicts,
                condition=FilterCondition.OR
            )
        )
        response = query_engine.query(query)
        return response
        
    
    vector_query_tool = FunctionTool.from_defaults(
        name=f"vector_tool_{name}",
        fn=vector_query
    )
    
    summary_index = SummaryIndex(nodes)
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
    )
    summary_tool = QueryEngineTool.from_defaults(
        name=f"summary_tool_{name}",
        query_engine=summary_query_engine,
        description=(
            "Use ONLY IF you want to get a holistic summary of MetaGPT. "
            "Do NOT use if you have specific questions over MetaGPT."
        ),
    )

    return vector_query_tool, summary_tool

In [5]:

vector_tool, summary_tool = get_doc_tools("budgetpdf.pdf", "budget")

In [6]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [7]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

In [8]:
agent_worker =  FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool],
    llm=llm
)

In [9]:
agent = AgentRunner(agent_worker)

In [11]:
print(agent.chat("what are the main highlights of interim budget"))

assistant: The main highlights of the interim budget include a focus on social justice with upliftment of major groups like the poor, women, youth, and farmers through various schemes such as PM-KISAN SAMMAN Yojana, PM Awas Yojana, and Ayushman Bharat. Investments in infrastructure, agriculture, and research for growth and employment were emphasized. Tax reforms were addressed with no changes proposed in tax rates and efforts to improve tax payer services. The budget projected a 7.3% growth in real GDP for FY 2023-24 and outlined a fiscal deficit of 5.1% of GDP for 2024-25. Additionally, there was a significant boost to capital expenditure, lower borrowings by the central government to facilitate more credit availability for the private sector, and a focus on boosting private investment in sunrise technologies and green growth initiatives. Empowerment of women through entrepreneurship and education, empowerment of farmers, boosting the agricultural sector, infrastructure development, e

In [12]:
agent.chat("what are the years mentioned in the above result?")

AgentChatResponse(response='assistant: The years mentioned in the above result are FY 2023-24 and 2024-25.', sources=[], source_nodes=[], is_dummy_stream=False)

In [13]:
task = agent.create_task("what are the main highlights of interim budget"
                         "and what are the years mentioned")

In [15]:
task.task_id

'3cdca868-55ec-44ca-b615-ec39384a51be'

In [16]:
step_output = agent.run_step(task.task_id)

In [17]:
step_output

TaskStepOutput(output=AgentChatResponse(response='assistant: None', sources=[ToolOutput(content='The main highlights of the interim budget include a significant focus on social justice with upliftment programs for the poor, women, youth, and farmers. Key initiatives such as financial assistance to farmers, credit support to street vendors, and schemes for vulnerable tribal groups were emphasized. Additionally, the budget outlined advancements in women empowerment, education, housing, healthcare, agriculture, infrastructure development, and tax reforms. There is also a strong emphasis on capital expenditure, fiscal deficit targets, FDI inflow, interest-free loans for states, boosting private investment in sunrise technologies, promoting green growth and renewable energy initiatives, enhancing the e-vehicle ecosystem, empowering women through entrepreneurship and education, supporting farmers through various schemes, boosting youth engagement in sports and technology, developing domestic

In [18]:
completed_steps = agent.get_completed_steps(task.task_id)

In [20]:
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 3cdca868-55ec-44ca-b615-ec39384a51be: 1
The main highlights of the interim budget include a significant focus on social justice with upliftment programs for the poor, women, youth, and farmers. Key initiatives such as financial assistance to farmers, credit support to street vendors, and schemes for vulnerable tribal groups were emphasized. Additionally, the budget outlined advancements in women empowerment, education, housing, healthcare, agriculture, infrastructure development, and tax reforms. There is also a strong emphasis on capital expenditure, fiscal deficit targets, FDI inflow, interest-free loans for states, boosting private investment in sunrise technologies, promoting green growth and renewable energy initiatives, enhancing the e-vehicle ecosystem, empowering women through entrepreneurship and education, supporting farmers through various schemes, boosting youth engagement in sports and technology, developing domestic tourism, improving tax collection

In [21]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 3cdca868-55ec-44ca-b615-ec39384a51be: 1


TaskStep(task_id='3cdca868-55ec-44ca-b615-ec39384a51be', step_id='dd9496e8-d3e5-4d35-96a6-55215ac74e64', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)